In [12]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import re
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import glob
from bs4 import BeautifulSoup
import re
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
from statsmodels.tsa.seasonal import seasonal_decompose
from matplotlib import pyplot

import matplotlib.pyplot as plt

In [ ]:
path = "/Users/macbook/Downloads/Dorado del mar/"
files = [f for f in glob.glob(path + "*.html")]
files = sorted(files)

In [ ]:
table_sub = pd.DataFrame([])
table_head = pd.DataFrame([])
for url in files:
    page = open(url)
    soup = BeautifulSoup(page.read())
    parsed = soup.findAll('tr', {'class': re.compile(r'extrarow extraroddrow|extrarow extraevenrow')})
    
    #subheaders
    for i in range(0, len(parsed)):
        td = parsed[i].findAll('td')
        td = re.sub("\n+|\r+|\t+", ";",  td[0].text)
        td = td.replace(' ;; ;;;',';').replace('\xa0','')
        td = re.sub("^;", "",  td)
        td = re.sub('[ ]{2,}',' ',  td)
        td = re.sub(";+", ";", td)
    
        cols_list = td.split(';')
        cols_dict={}
    
        for j in range(0, len(cols_list)-1,2):
            cols_dict[cols_list[j]] = cols_list[j+1]
        row = pd.DataFrame(cols_dict, index=[i])
        table_sub = pd.concat([table_sub, row])
    
    #headers
    prop_id = soup.findAll('td', attrs={'class':"evencol tablecol fld_community"})
    sale_date = soup.findAll('td', attrs={'class':"oddcol tablecol fld_settled_sale_date"})
    sale_price = soup.findAll('td', attrs={'class':"oddcol tablecol fld_sales_price"})
    appr_value = soup.findAll('td', attrs={'class':"evencol tablecol fld_appraisal_value"})
    
    col_list =[[],[],[],[]]
    col_list_test = [prop_id, sale_date, sale_price, appr_value]
    for n,j in enumerate(col_list_test):
        for i in range(0, len(prop_id)):
            row = re.sub("\n+|\r+|\t+", ";",  j[i].text)
            row = re.sub(";+", ";",  row)
            row = re.sub("^;|;$", "",  row)
            col_list[n].append(row)
    
    prop_id_col = col_list[0]
    sale_date_col = col_list[1]
    sale_price_col = col_list[2]
    appr_value_col = col_list[3]
    prop_type = [x.split(';')[1] for x in prop_id_col]
    prop_id = [x.split(';')[2] for x in prop_id_col]
    community = [x.split(';')[0] for x in prop_id_col]
    table_page = pd.DataFrame({'Community':community,
                          'Building type':prop_type,
                          'Property id':prop_id,
                          'Sale date':sale_date_col,
                          'Sale price':sale_price_col,
                          'Appraisal value':appr_value_col})
    table_head = pd.concat([table_head, table_page])

In [ ]:
table_head = table_head.reset_index(drop=True)
table_sub = table_sub.reset_index(drop=True)
del_mar_full = pd.concat([table_head,table_sub], axis=1)
del_mar_full.shape

In [ ]:
del_mar_full.to_csv('del_mar_full.csv')

In [2]:
del_mar_full = pd.read_csv('del_mar_full.csv', index_col=[0])

## Postprocessing

In [3]:
cols = ['Municipality','Community', 'Property id', 'Address/Unit', 'Street','GLA(feet)', 'Rooms', 
        'Bedrooms', 'Bathrooms','Full bath', 'Half bath', 'Site(meters)', 'Garage', 'Stories', 'Units',  
        'Building type', 'Cons. Type','Conditions','Actual age', 'Efective Age',  'Extras', 'Sale price',   
        'Sale date','Appraisal value', 'Appraisal date',  'Opinion of site value', 'Marketing time',  
        'Days on market', 'Prior sale date', 'Prior sale price', 'Latitud','Longitud','Zoning'
        ]

In [4]:
del_mar_full = del_mar_full[cols]

In [5]:
# id
del_mar_full['Property id'] = del_mar_full['Property id'].str.split('ID: ', expand=True)[1]
# address
del_mar_full['Address/Unit'] = del_mar_full['Street'] + ', ' + del_mar_full['Address/Unit']
# price
for col in ['Sale price', 'Appraisal value','Opinion of site value','Prior sale price','GLA(feet)']:
    del_mar_full[col] = del_mar_full[col].str.replace('\$','').str.replace(',','')
    del_mar_full[col] = del_mar_full[col].replace({'':np.nan})
# to floats
for col in ['Bedrooms','Sale price', 'Appraisal value','Opinion of site value','Prior sale price'
           ,'Bathrooms','GLA(feet)']:
    del_mar_full[col] = del_mar_full[col].astype('float')
del_mar_full['Sale date'] = pd.to_datetime(del_mar_full['Sale date'])

## Comps

In [6]:
comps = del_mar_full[(del_mar_full['Bedrooms'] >=2) & 
                     (del_mar_full['Bathrooms'] >=2) &
                     (del_mar_full['GLA(feet)'] >=2000)].reset_index(drop=True)

In [7]:
del_mar_full.shape

(231, 33)

In [8]:
comps.shape

(96, 33)

In [9]:
comps['Sale date Year'] = comps['Sale date'].apply(lambda x: x.year)
comps['Sale date Quarter'] = comps['Sale date'].apply(lambda x: x.quarter)
comps['Sale date Year-Quarter'] = comps['Sale date Year'].astype(str) +' Q'+ comps['Sale date Quarter'].astype(str)
del_mar_full['Sale date Year'] = del_mar_full['Sale date'].apply(lambda x: x.year)
del_mar_full['Sale date Quarter'] = del_mar_full['Sale date'].apply(lambda x: x.quarter)
del_mar_full['Sale date Year-Quarter'] = del_mar_full['Sale date Year'].astype(str) +' Q'+ del_mar_full['Sale date Quarter'].astype(str)

# Plots for comps

In [14]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=del_mar_full['Sale price'], xbins_size=30000,  name='All houses'))
fig.add_trace(go.Histogram(x=comps['Sale price'], xbins_size=30000, name='Comps'))

fig.update_layout(title_text='Price distribution of the sold houses',
                  width=600, height=400, plot_bgcolor='rgba(0,0,0,0)', showlegend=True,
                  yaxis=dict(gridcolor='#EEEEEE', nticks=18),
                  xaxis=dict(gridcolor='#EEEEEE', nticks=18,
                             tickfont=dict(size=10)))
fig.update_layout(barmode='overlay')
fig.update_traces(opacity=0.8)
fig.show()

In [15]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=del_mar_full[del_mar_full['GLA(feet)'] > 0]['GLA(feet)'], xbins_size=150,  name='All houses'))
fig.add_trace(go.Histogram(x=comps['GLA(feet)'], xbins_size=150,  name='Comps'))

fig.update_layout(title_text='GLA distribution of the sold houses',
                  width=600, height=400, plot_bgcolor='rgba(0,0,0,0)', showlegend=True,
                  yaxis=dict(gridcolor='#EEEEEE', nticks=10),
                  xaxis=dict(gridcolor='#EEEEEE', nticks=18,
                             tickfont=dict(size=10)))
fig.update_layout(barmode='overlay')
fig.update_traces(opacity=0.8)
fig.show()

In [16]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=del_mar_full[del_mar_full['GLA(feet)'] > 0]['Bathrooms'], xbins_size=1,  name='All houses'))
fig.add_trace(go.Histogram(x=comps['Bathrooms'], xbins_size=1,  name='Comps'))

fig.update_layout(title_text='baths distribution of the sold houses',
                  width=400, height=400, plot_bgcolor='rgba(0,0,0,0)', showlegend=True,
                  yaxis=dict(gridcolor='#EEEEEE', nticks=5),
                  xaxis=dict(gridcolor='#EEEEEE', nticks=6,
                             tickfont=dict(size=12)))
fig.update_layout(barmode='overlay')
fig.update_traces(opacity=0.7)
fig.show()

In [17]:
fig = go.Figure()
fig.add_trace(go.Scatter(x= comps['GLA(feet)'], 
                         y= comps['Sale price'],
                         mode='markers',
                         marker_color='blue',
                         marker_size=12,
                         marker_opacity=0.3,
                         name = 'Min Quarter value',
                        ),
             )
fig.update_layout(title_text='Price change Dorado del Mar 2008-2019',
                  width=700, height=400, plot_bgcolor='rgba(0,0,0,0)', showlegend=True,
                  yaxis=dict(gridcolor='#EEEEEE', nticks=10),
                  xaxis=dict(nticks=10, gridcolor='#EEEEEE', 
                             tickfont=dict(size=12)))
fig.show()

## pricy

In [18]:
pricy = del_mar_full[del_mar_full['Sale price'] >=250000].reset_index(drop=True)

In [19]:
fig = make_subplots(rows=1, cols=3,horizontal_spacing = 0.02,column_widths=[0.4, 0.23,0.23], shared_yaxes=True)

fig.add_trace(go.Histogram(x=pricy[pricy['GLA(feet)'] > 0]['GLA(feet)'], xbins_size=300,  name='All houses'),1,1)
fig.add_trace(go.Histogram(x=pricy['Bedrooms'], xbins_size=1,  name='All houses'),1,2)
fig.add_trace(go.Histogram(x=pricy['Bathrooms'], xbins_size=1,  name='All houses'),1,3)



fig.update_layout(title_text='GLA distribution of the sold houses',
                  width=900, height=500, plot_bgcolor='rgba(0,0,0,0)', showlegend=False,
                  yaxis=dict(gridcolor='#EEEEEE', nticks=5),
                  xaxis=dict(gridcolor='#EEEEEE', nticks=18,
                             tickfont=dict(size=12)))
axes_names = ['Area','', 'Bedrooms', 'Bathrooms','','']
for j,i in enumerate(['xaxis','yaxis','xaxis2','xaxis3','yaxis2','yaxis3']):
    fig.layout[i]['showgrid'] = True
    fig.layout[i]['gridwidth'] = 0.1
    fig.layout[i]['showgrid'] = True
    fig.layout[i]['gridcolor'] = '#EEEEEE'
    
    #fig.layout[i]['nticks'] = 15
    if 'xaxis' in i:
        fig.layout[i]['title'] = axes_names[j]
    
fig.update_layout(barmode='overlay')
fig.update_traces(opacity=0.8)
fig.show()

In [20]:
fig = make_subplots(rows=1, cols=2,horizontal_spacing = 0.08,
                    column_widths=[0.5, 0.5], 
                    shared_yaxes=True)

fig.add_trace(go.Bar(x=stories.index,
                     y=stories['Stories'].values,  name='All houses'),1,1)
fig.add_trace(go.Histogram(x=pricy['Actual age'], xbins_size=1,  name='Ag'),1,2)



fig.update_layout(title_text='GLA distribution of the sold houses',
                  bargap=0.3,
                  width=900, height=500, plot_bgcolor='rgba(0,0,0,0)', showlegend=False,
                  yaxis=dict(gridcolor='#EEEEEE', nticks=5),
                  xaxis=dict(gridcolor='#EEEEEE',
                             tickfont=dict(size=12)))
axes_names = ['Stories','', 'Age', 'Age','']
for j,i in enumerate(['xaxis','yaxis','xaxis2','yaxis2']):
    fig.layout[i]['showgrid'] = True
    fig.layout[i]['gridwidth'] = 0.1
    fig.layout[i]['showgrid'] = True
    fig.layout[i]['gridcolor'] = '#EEEEEE'

    if 'xaxis' in i:
        fig.layout[i]['title'] = axes_names[j]
    
fig.update_traces(opacity=0.8)
fig.show()

NameError: name 'stories' is not defined

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x= pricy['Sale date'], 
                         y= pricy['Sale price'],
                         mode='markers',
                         name = 'Dorado del Mar',
                         marker=dict(size=10, opacity=0.6)
                           ))

fig.update_layout(title_text='Price per Sq.ft. Dorado del Mar Change 2008-2019',
                  width=800, height=300, plot_bgcolor='rgba(0,0,0,0)', showlegend=True,
                  yaxis=dict(gridcolor='#EEEEEE'),
                  xaxis=dict(tickangle=-45, nticks=50, gridcolor='#EEEEEE', 
                             tickfont=dict(size=9)))

fig.show()

### sales volume

In [ ]:
sales_volume_mar_comps = comps.groupby('Sale date Year-Quarter')['Sale price'].count()
sales_volume_mar_comps = pd.DataFrame(sales_volume_mar_comps ).rename(columns={'Sale price':'Volume'})

del_mar_full = del_mar_full.sort_values('Sale date Year-Quarter')
sales_volume_dorado = del_mar_full.groupby('Sale date Year-Quarter')['Sale price'].count()
sales_volume_dorado = pd.DataFrame(sales_volume_dorado).rename(columns={'Sale price':'Volume'})


#sales_volume['Sale price'] = [x/400*100 for x in sales_volume['Sale price']]

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x= sales_volume_mar_comps.index, 
                         y= sales_volume_mar_comps['Volume'],
                         mode='markers',
                         name = 'Dorado del Mar',
                         marker=dict(size=10, opacity=0.6)
                           ))
fig.add_trace(go.Scatter(x= sales_volume_dorado.index, 
                         y= sales_volume_dorado['Volume'],
                         mode='markers',
                         name = 'Dorado',
                         marker=dict(size=10, opacity=0.6)
                           ))



fig.update_layout(title_text='Price per Sq.ft. Dorado del Mar Change 2008-2019',
                  width=1200, height=300, plot_bgcolor='rgba(0,0,0,0)', showlegend=True,
                  yaxis=dict(gridcolor='#EEEEEE', nticks=10),
                  xaxis=dict(tickangle=-45, gridcolor='#EEEEEE', 
                             tickfont=dict(size=9)))

fig.show()

# comps grouping

In [21]:
comps_quarters = comps.groupby('Sale date Year-Quarter')['Sale price'].agg(Price_min='min', 
                                                                          Price_max='max', 
                                                                          Price_median='median')
comps_quarters = pd.DataFrame(comps_quarters)

In [22]:
comps_quarters['Price_min'].mean()

224558.97435897434

In [23]:
min_change = [(comps_quarters['Price_min'][x+1]-comps_quarters['Price_min'][x])/comps_quarters['Price_min'][x]*100 for x in range(comps_quarters.shape[0]-1)]
percent_min = [0]
percent_min.extend(min_change)
max_change = [(comps_quarters['Price_max'][x+1]-comps_quarters['Price_max'][x])/comps_quarters['Price_max'][x]*100 for x in range(comps_quarters.shape[0]-1)]
percent_max = [0]
percent_max.extend(max_change)
comps_quarters['Price_max_change'] = percent_max
comps_quarters['Price_min_change'] = percent_min

In [24]:
def set_annotation(df, x, y, k, min_or_max, percent=False, fontsize=6,): 
    if min_or_max == 'min':
        col_one = 'Price_min_change'
        col_two = 'Price_min'
        ay = 50
        #ax=-10
    elif min_or_max == 'max':
        col_one = 'Price_max_change'
        col_two = 'Price_max'
        ay = -60
        #ax=-10
    if df[col_one][k] > 0:
        color='green'
        prefix = '+'
        ay = ay+10
    else:
        color='brown'
        prefix = ''
    
    sub=1000
    K = 'K'
    if percent == True:
        sub = 1
        K = ''
    value = df[col_two][k]/sub
    return dict(x=x,  
                y=y,       
                text='{:.0f}{}<br>{}{:.0f}%'.format(value,K, prefix,
                                                   df[col_one][k]),
                font=dict(size=fontsize, color=color),
                showarrow=True,
                arrowhead=0,
                #xref="x",
                #yref="y",
                ax=0,
                ay=ay,
                #ayref='y',
                arrowwidth=0.5,
                arrowcolor="#616161",
                opacity=1
               ) 

In [25]:
annotations_min=[set_annotation(comps_quarters, k, comps_quarters['Price_min'][k], k, 'min') for k in range(comps_quarters.shape[0])]
annotations_max=[set_annotation(comps_quarters, k, comps_quarters['Price_max'][k], k, 'max') for k in range(comps_quarters.shape[0])]
annotations_min.extend(annotations_max)

In [26]:
fig = go.Figure()
fig.add_trace(go.Scatter(x= comps_quarters.index, 
                         y= comps_quarters['Price_min'],
                         mode='lines+markers',
                         marker_color='blue',
                         name = 'Min Quarter value',
                        ),
             )
fig.add_trace(go.Scatter(x= comps_quarters.index, 
                         y= comps_quarters['Price_max'],
                         mode='lines+markers',
                         marker_color='red',
                         name = 'Max Quarter value',
                         fill='tonexty',fillcolor='rgba(248, 245, 159, 0.1)'))
fig.add_trace(go.Scatter(x= comps_quarters.index, 
                         y= comps_quarters['Price_median'],
                         mode='lines',
                         name = 'Median Quarter value',
                         line = dict(shape = 'spline', width=1, dash='dashdot', color='grey')
                           ))
fig.update_layout(title_text='Price change Dorado del Mar 2008-2019',
                  width=1000, height=500, plot_bgcolor='rgba(0,0,0,0)', showlegend=True,
                  yaxis=dict(gridcolor='#EEEEEE', nticks=18),
                  xaxis=dict(tickangle=-45, gridcolor='#EEEEEE', 
                             tickfont=dict(size=9)))
fig.update_layout(annotations=annotations_min)
fig.show()

### per sq.m

In [27]:
comps['Price/area'] = comps['Sale price']/comps['GLA(feet)']

In [28]:
comps_q_sq = comps.groupby('Sale date Year-Quarter')['Price/area'].agg(Price_min='min', 
                                                                          Price_max='max', 
                                                                          Price_median='median')
comps_q_sq = pd.DataFrame(comps_q_sq)

In [29]:
min_change = [(comps_q_sq['Price_min'][x+1]-comps_q_sq['Price_min'][x])/comps_q_sq['Price_min'][x]*100 for x in range(comps_q_sq.shape[0]-1)]
percent_min = [0]
percent_min.extend(min_change)
max_change = [(comps_q_sq['Price_max'][x+1]-comps_q_sq['Price_max'][x])/comps_q_sq['Price_max'][x]*100 for x in range(comps_q_sq.shape[0]-1)]
percent_max = [0]
percent_max.extend(max_change)
comps_q_sq['Price_max_change'] = percent_max
comps_q_sq['Price_min_change'] = percent_min

In [30]:
(84-64)/64*100

31.25

In [31]:
annotations_min=[set_annotation(comps_q_sq, k, comps_q_sq['Price_min'][k], k, 'min', percent=True) for k in range(comps_q_sq.shape[0])]
annotations_max=[set_annotation(comps_q_sq, k, comps_q_sq['Price_max'][k], k, 'max', percent=True) for k in range(comps_q_sq.shape[0])]
annotations_min.extend(annotations_max)

In [32]:
fig = go.Figure()
fig.add_trace(go.Scatter(x= comps_q_sq.index, 
                         y= comps_q_sq['Price_min'],
                         mode='lines+markers',
                         name = 'Min Quarter value'
                           ))
fig.add_trace(go.Scatter(x= comps_q_sq.index, 
                         y= comps_q_sq['Price_max'],
                         mode='lines+markers',
                         name = 'Max Quarter value',
                         fill='tonexty',fillcolor='rgba(248, 245, 159, 0.1)'
                           ))
fig.add_trace(go.Scatter(x= comps_q_sq.index, 
                         y= comps_q_sq['Price_median'],
                         mode='lines',
                         name = 'Median Quarter value',
                         line = dict(shape = 'spline', width=1, dash='dashdot', color='grey')
                           ))
fig.update_layout(title_text='Price per Sq.ft. Dorado del Mar Change 2008-2019',
                  width=1000, height=500, plot_bgcolor='rgba(0,0,0,0)', showlegend=True,
                  yaxis=dict(gridcolor='#EEEEEE', nticks=18),
                  xaxis=dict(tickangle=-45, gridcolor='#EEEEEE', 
                             tickfont=dict(size=9)))
fig.update_layout(annotations=annotations_min)


fig.show()

## Forecasting

In [ ]:
plot = comps.set_index('Sale date')
plot = plot.sort_values('Sale date')
decomposition = seasonal_decompose(plot['Sale price'], model='multiplicative', freq=12)
plt.figure(figsize=(20,10))
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

plt.subplot(411)
plt.plot(plot['Sale price'], color = "#00b8ff", label = "Original")
plt.legend(loc='best')
plt.subplot(412)
plt.plot(trend, 'b', label = "Trend")
plt.legend(loc='best')
plt.subplot(413)
plt.plot(seasonal, color = "#ff00ff", label = "Seasonality")
plt.legend(loc='left top')
plt.subplot(414)
plt.plot(residual, 'y', label = "Residual")
plt.legend(loc='best')
plt.tight_layout()
plt.show();

### model

In [ ]:
from statsmodels.tsa.stattools import adfuller
import itertools
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
def evaluate_arima_model(params, step):
    model = ARIMA(data_arima['Price_min'], order = params).fit()
    pred = model.forecast(step)[0]
    pred = pd.DataFrame(pred, columns=['Predict'])
    test_data['Forecast'] = pred['Predict'].values
    
    mape = mean_absolute_percentage_error(test_data['Price_min'], test_data['Forecast'])
    
    return mape

In [ ]:
def evaluate_models(step, val_range):
    best_score, best_params = float("inf"), None

    p = d = q = range(val_range)
    pdq = list(itertools.product(p, d, q))
    for params in pdq:
        try:
            mape = evaluate_arima_model(params, step)
            if mape < best_score:
                best_score, best_params = mape, params
                print('ARIMA % % MAPE=%.3f' % (params, mape))
        except:
            continue
    print(best_params, best_score)

In [ ]:
def compare_plots(forecast, original, interval):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x= interval.index, 
                             y= forecast.values,
                             mode='lines',
                             name = 'Predict'
                               ))
    fig.add_trace(go.Scatter(x= interval.index, 
                             y= original.values,
                             mode='lines',
                             name = 'Original',
                               ))

    fig.update_layout(title_text='Predicted vs. Original Plots',
                      width=800, height=400, plot_bgcolor='rgba(0,0,0,0)', showlegend=True,
                      yaxis=dict(gridcolor='#EEEEEE', nticks=18),
                      xaxis=dict(tickangle=-45, gridcolor='#EEEEEE', 
                                 tickfont=dict(size=9), nticks=50))

    fig.show()

### quarterly

In [ ]:
comps_q = comps[['Sale price','Sale date']]
comps_q = comps_q.set_index('Sale date')
comps_q_sq = comps[['Price/area','Sale date']]
comps_q_sq = comps_q_sq.set_index('Sale date')

In [ ]:
comps_q.index = pd.DatetimeIndex(comps_q.index).to_period("Q")
comps_q_sq.index = pd.DatetimeIndex(comps_q_sq.index).to_period("Q")

In [ ]:
comps_q = comps_q.groupby(pd.Grouper(freq='Q'))['Sale price'].agg(Price_min='min',
                                               Price_max='max',
                                               Price_median='median')

In [ ]:
comps_q_sq = comps_q_sq.groupby(pd.Grouper(freq='Q'))['Price/area'].agg(Price_min='min',
                                               Price_max='max',
                                               Price_median='median')

In [ ]:
# filling missed values
import math
def fill_missed(data):
    for i in range(0,data.shape[0]-1):
        price_min = data['Price_min'][data['Price_min'].index[i]]
        pred_min = data['Price_min'][data['Price_min'].index[i-1]]
        next_min = data['Price_min'][data['Price_min'].index[i+1]]
    
        if math.isnan(price_min) == True:
            price_min = (pred_min+next_min)/2
            data['Price_min'][data['Price_min'].index[i]] = price_min
    
        price_max = data['Price_max'][data['Price_max'].index[i]]
        pred_max = data['Price_max'][data['Price_max'].index[i-1]]
        next_max = data['Price_max'][data['Price_max'].index[i+1]]
    
        if math.isnan(price_max) == True:
            price_max = (pred_max + next_max)/2
            data['Price_max'][data['Price_max'].index[i]] = price_max

    for i in range(0,data.shape[0]-1):
        price_median = data['Price_median'][data['Price_median'].index[i]]
        pred_median = data['Price_median'][data['Price_median'].index[i-1]]
        next_median = data['Price_median'][data['Price_median'].index[i+1]]
    
        if math.isnan(price_median) == True:
            price_median = (pred_median+next_median)/2
            data['Price_median'][data['Price_median'].index[i]] = price_median
    return data

In [ ]:
comps_q = fill_missed(comps_q)
comps_q_sq = fill_missed(comps_q_sq)

### monthly

In [ ]:
comps_months = comps[['Sale price','Sale date']]
comps_months = comps_months.set_index('Sale date')
comps_months = comps_months.groupby(pd.Grouper(freq='M'))['Sale price'].agg(Price_min='min',
                                               Price_max='max',
                                               Price_median='median')
comps_months_sq = comps[['Price/area','Sale date']]
comps_months_sq = comps_months_sq.set_index('Sale date')
comps_months_sq = comps_months_sq.groupby(pd.Grouper(freq='M'))['Price/area'].agg(Price_min='min',
                                               Price_max='max',
                                               Price_median='median')

In [ ]:
def missed_months(data, pred_q):
    for i in range(0,data.shape[0]-1):
        price_min = data['Price_min'][data['Price_min'].index[i]]
        pred_min = data['Price_min'][data['Price_min'].index[i-1]]
        next_min = data['Price_min'][data['Price_min'].index[i+1]]
    
        if math.isnan(pred_min) == True or math.isnan(next_min) == True:
            year = data['Price_min'].index[i].year
            quarter = data['Price_min'].index[i].quarter
            index = str(year)+'Q'+str(quarter)
            price_min = pred_q['Price_min'][index]
            data['Price_min'][data['Price_min'].index[i]] = price_min
        #print (price_min)
        else:
            price_min = (pred_min+next_min)/2
            data['Price_min'][data['Price_min'].index[i]] = price_min
        
    for i in range(0,data.shape[0]-1):
        price_max = data['Price_max'][data['Price_max'].index[i]]
        pred_max = data['Price_max'][data['Price_max'].index[i-1]]
        next_max = data['Price_max'][data['Price_max'].index[i+1]]
    
        if math.isnan(pred_max) == True or math.isnan(next_max) == True:
            year = data['Price_max'].index[i].year
            quarter = data['Price_max'].index[i].quarter
            index = str(year)+'Q'+str(quarter)
            price_max = pred_q['Price_max'][index]
            data['Price_max'][data['Price_max'].index[i]] = price_max
            #print (price_max)
        else:
            price_max = (pred_max+next_max)/2
            data['Price_max'][data['Price_max'].index[i]] = price_max

    for i in range(0,data.shape[0]-1):
        price_median = data['Price_median'][data['Price_median'].index[i]]
        pred_median = data['Price_median'][data['Price_median'].index[i-1]]
        next_median = data['Price_median'][data['Price_median'].index[i+1]]
    
        if math.isnan(pred_median) == True or math.isnan(next_median) == True:
            year = data['Price_median'].index[i].year
            quarter = data['Price_median'].index[i].quarter
            index = str(year)+'Q'+str(quarter)
            price_median = pred_q['Price_median'][index]
            data['Price_median'][data['Price_median'].index[i]] = price_median
            #print (price_median)
        else:
            price_median = (pred_median+next_median)/2
            data['Price_median'][data['Price_median'].index[i]] = price_median
    return data

In [ ]:
comps_months = missed_months(comps_months, comps_q)

In [ ]:
comps_months_sq = missed_months(comps_months_sq, comps_q_sq)

### prophet

In [ ]:
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
from fbprophet.plot import plot_cross_validation_metric

In [ ]:
steps = 72
test_interval = steps #months
data_fb = comps_months[0:-test_interval]
data_fb.drop(columns=['Price_max','Price_min'], inplace=True)
test_data = comps_months[-test_interval:]
test_data.drop(columns=['Price_max','Price_min'], inplace=True)
data_fb = data_fb.rename(columns={'Price_median':'y'})
data_fb['ds'] = data_fb.index
test_data = test_data.rename(columns={'Price_median':'y'})
test_data['ds'] = test_data.index

In [ ]:
whole_interval = pd.concat([data_fb, test_data], axis=0)

In [ ]:
m = Prophet(weekly_seasonality=False, daily_seasonality=False, yearly_seasonality=True)
m.add_seasonality(name='yearly', period=260, fourier_order=5)
m.fit(data_fb)
future = m.make_future_dataframe(periods=steps, freq='M')
forecast = m.predict(future)
pred = forecast['yhat'].tail(n=test_interval).values
#pred = pred.predicted_mean
pred = pd.DataFrame(pred, columns=['Predict'])
test_data['Forecast'] = pred['Predict'].values
print(mean_absolute_percentage_error(test_data['y'], test_data['Forecast']))
compare_plots(test_data['Forecast'], test_data['y'], test_data)

In [ ]:
m = Prophet(weekly_seasonality=False, daily_seasonality=False, yearly_seasonality=True)
m.add_seasonality(name='yearly', period=260, fourier_order=5)
m.fit(whole_interval)
future = m.make_future_dataframe(periods=steps, freq='M')
forecast = m.predict(future)
pred = forecast['yhat'].tail(n=test_interval).values
pred = pd.DataFrame(pred, columns=['y'])
pred.index = pd.date_range('2019-03', periods=steps, freq='M')
#test_data['Forecast'] = pred['Predict'].values
#print(mean_absolute_percentage_error(test_data['y'], test_data['Forecast']))
#compare_plots(test_data['Forecast'], test_data['y'], test_data)

In [ ]:
hist_pred = pd.concat([whole_interval, pred], axis=0) 

In [ ]:
comps_q_whole = hist_pred.copy()

comps_q_whole.index = pd.DatetimeIndex(comps_q_whole.index).to_period("Q")

comps_q_whole = comps_q_whole.groupby(pd.Grouper(freq='Q'))['y'].agg(Price_min='min',
                                               Price_max='max',
                                               Price_median='median')

In [ ]:
min_change = [(comps_q_whole['Price_min'][x+1]-comps_q_whole['Price_min'][x])/comps_q_whole['Price_min'][x]*100 for x in range(comps_q_whole.shape[0]-1)]
percent_min = [0]
percent_min.extend(min_change)
max_change = [(comps_q_whole['Price_max'][x+1]-comps_q_whole['Price_max'][x])/comps_q_whole['Price_max'][x]*100 for x in range(comps_q_whole.shape[0]-1)]
percent_max = [0]
percent_max.extend(max_change)
comps_q_whole['Price_max_change'] = percent_max
comps_q_whole['Price_min_change'] = percent_min

In [ ]:
annotations_min=[set_annotation(comps_q_whole, k, comps_q_whole['Price_min'][k], k, 'min') for k in range(comps_q_whole.shape[0])]
annotations_max=[set_annotation(comps_q_whole, k, comps_q_whole['Price_max'][k], k, 'max') for k in range(comps_q_whole.shape[0])]
annotations_min.extend(annotations_max)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x= comps_q_whole.loc[:'2019Q1'].index.astype(str), 
                             y= comps_q_whole.loc[:'2019Q1']['Price_min'].values,
                             mode='lines',
                             name = 'Original'
                               ))
fig.add_trace(go.Scatter(x= comps_q_whole.loc[:'2019Q1'].index.astype(str), 
                             y= comps_q_whole.loc[:'2019Q1']['Price_max'].values,
                             mode='lines',
                             name = 'Original'
                               ))

fig.add_trace(go.Scatter(x= comps_q_whole.loc['2019Q1':].index.astype(str), 
                             y= comps_q_whole.loc['2019Q1':]['Price_max'].values,
                             mode='lines',
                          line = dict(dash='dash', color='red'),
                             name = 'lower_bond',
                               ))
fig.add_trace(go.Scatter(x= comps_q_whole.loc['2019Q1':].index.astype(str), 
                             y= comps_q_whole.loc['2019Q1':]['Price_min'].values,
                             mode='lines',
                          line = dict(dash='dash', color='blue'),
                             name = 'lower_bond',
                               ))

fig.update_layout(title_text='Predicted vs. Original Plots',
                      width=1100, height=500, plot_bgcolor='rgba(0,0,0,0)', showlegend=True,
                      yaxis=dict(gridcolor='#EEEEEE', nticks=10),
                      xaxis=dict(tickangle=-45, gridcolor='#EEEEEE', 
                                 tickfont=dict(size=9), nticks=50))
fig.update_layout(annotations=annotations_min)

fig.show()

In [ ]:
2900*80

## forecast per sq.ft

In [ ]:
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
from fbprophet.plot import plot_cross_validation_metric

steps = 24
test_interval = steps #months
data_fb = comps_months_sq[0:-test_interval]
data_fb.drop(columns=['Price_max','Price_min'], inplace=True)
test_data = comps_months_sq[-test_interval:]
test_data.drop(columns=['Price_max','Price_min'], inplace=True)
data_fb = data_fb.rename(columns={'Price_median':'y'})
data_fb['ds'] = data_fb.index
test_data = test_data.rename(columns={'Price_median':'y'})
test_data['ds'] = test_data.index

whole_interval = pd.concat([data_fb, test_data], axis=0)

m = Prophet(weekly_seasonality=False, daily_seasonality=False, yearly_seasonality=True)
m.add_seasonality(name='yearly', period=260, fourier_order=5)
m.fit(data_fb)
future = m.make_future_dataframe(periods=steps, freq='M')
forecast = m.predict(future)
pred = forecast['yhat'].tail(n=test_interval).values
#pred = pred.predicted_mean
pred = pd.DataFrame(pred, columns=['Predict'])
test_data['Forecast'] = pred['Predict'].values
print(mean_absolute_percentage_error(test_data['y'], test_data['Forecast']))
compare_plots(test_data['Forecast'], test_data['y'], test_data)

In [ ]:
m = Prophet(weekly_seasonality=False, daily_seasonality=False, yearly_seasonality=True)
m.add_seasonality(name='yearly', period=260, fourier_order=5)
m.fit(whole_interval)
future = m.make_future_dataframe(periods=steps, freq='M')
forecast = m.predict(future)
pred = forecast['yhat'].tail(n=test_interval).values
pred = pd.DataFrame(pred, columns=['y'])
pred.index = pd.date_range('2019-03', periods=steps, freq='M')

In [ ]:
hist_pred = pd.concat([whole_interval, pred], axis=0) 

comps_q_sq_whole = hist_pred.copy()

comps_q_sq_whole.index = pd.DatetimeIndex(comps_q_sq_whole.index).to_period("Q")

comps_q_sq_whole = comps_q_sq_whole.groupby(pd.Grouper(freq='Q'))['y'].agg(Price_min='min',
                                               Price_max='max',
                                               Price_median='median')

In [ ]:
min_change = [(comps_q_sq_whole['Price_min'][x+1]-comps_q_sq_whole['Price_min'][x])/comps_q_sq_whole['Price_min'][x]*100 for x in range(comps_q_sq_whole.shape[0]-1)]
percent_min = [0]
percent_min.extend(min_change)
max_change = [(comps_q_sq_whole['Price_max'][x+1]-comps_q_sq_whole['Price_max'][x])/comps_q_sq_whole['Price_max'][x]*100 for x in range(comps_q_sq_whole.shape[0]-1)]
percent_max = [0]
percent_max.extend(max_change)
comps_q_sq_whole['Price_max_change'] = percent_max
comps_q_sq_whole['Price_min_change'] = percent_min

annotations_min=[set_annotation(comps_q_sq_whole, k, comps_q_sq_whole['Price_min'][k], k, 'min', percent=True) for k in range(comps_q_sq_whole.shape[0])]
annotations_max=[set_annotation(comps_q_sq_whole, k, comps_q_sq_whole['Price_max'][k], k, 'max', percent=True) for k in range(comps_q_sq_whole.shape[0])]
annotations_min.extend(annotations_max)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x= comps_q_sq_whole.loc[:'2019Q1'].index.astype(str), 
                             y= comps_q_sq_whole.loc[:'2019Q1']['Price_min'].values,
                             mode='lines',
                             name = 'Original'
                               ))
fig.add_trace(go.Scatter(x= comps_q_sq_whole.loc[:'2019Q1'].index.astype(str), 
                             y= comps_q_sq_whole.loc[:'2019Q1']['Price_max'].values,
                             mode='lines',
                             name = 'Original'
                               ))

fig.add_trace(go.Scatter(x= comps_q_sq_whole.loc['2019Q1':].index.astype(str), 
                             y= comps_q_sq_whole.loc['2019Q1':]['Price_max'].values,
                             mode='lines',
                          line = dict(dash='dash', color='red'),
                             name = 'lower_bond',
                               ))
fig.add_trace(go.Scatter(x= comps_q_sq_whole.loc['2019Q1':].index.astype(str), 
                             y= comps_q_sq_whole.loc['2019Q1':]['Price_min'].values,
                             mode='lines',
                          line = dict(dash='dash', color='blue'),
                             name = 'lower_bond',
                               ))

fig.update_layout(title_text='Predicted vs. Original Plots',
                      width=1150, height=400, plot_bgcolor='rgba(0,0,0,0)', showlegend=True,
                      yaxis=dict(gridcolor='#EEEEEE', nticks=10),
                      xaxis=dict(tickangle=-45, gridcolor='#EEEEEE', 
                                 tickfont=dict(size=9), nticks=50))
fig.update_layout(annotations=annotations_min)

fig.show()

# *

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x= test_data.index, 
                             y= test_data['y'].values,
                             mode='lines',
                             name = 'Original'
                               ))
fig.add_trace(go.Scatter(x= test_data.index, 
                             y= forecast['yhat_lower'].tail(n=test_interval).values,
                             mode='lines',
                          line = dict(dash='dash'),
                             name = 'lower_bond',
                               ))

fig.add_trace(go.Scatter(x= test_data.index, 
                             y= forecast['yhat_upper'].tail(n=test_interval).values,
                             mode='lines',
                             line = dict(dash='dash'),
                         
                             name = 'upper_bond',
                               ))

fig.update_layout(title_text='Predicted vs. Original Plots',
                      width=800, height=400, plot_bgcolor='rgba(0,0,0,0)', showlegend=True,
                      yaxis=dict(gridcolor='#EEEEEE', nticks=18),
                      xaxis=dict(tickangle=-45, gridcolor='#EEEEEE', 
                                 tickfont=dict(size=9), nticks=50))

In [ ]:
df_cv = cross_validation(m, initial='1500 days', period='180 days', horizon = '720 days')
df_cv.head()

In [ ]:
df_p = performance_metrics(df_cv)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x= df_p['horizon'].values, 
                             y= df_p['mape'].values,
                             mode='lines',
                             name = 'Original'
                               ))

fig.update_layout(title_text='Predicted vs. Original Plots',
                      width=800, height=400, plot_bgcolor='rgba(0,0,0,0)', showlegend=True,
                      yaxis=dict(gridcolor='#EEEEEE', nticks=18),
                      xaxis=dict(tickangle=-45, gridcolor='#EEEEEE', 
                                 tickfont=dict(size=9), nticks=50))

In [ ]:
df_cv.shape, data_fb.shape, df_p.shape

# Dorado

In [ ]:
dorado = pd.read_csv('Dorado 2000-2020.csv')

## Postprocessing

In [ ]:
cols = ['Municipality','Community', 'Property id', 'Address/Unit', 'Street','GLA(feet)', 'Rooms', 
        'Bedrooms', 'Bathrooms','Full bath', 'Half bath', 'Site(meters)', 'Garage', 'Stories', 'Units',  
        'Building type', 'Cons. Type','Conditions','Actual age', 'Efective Age',  'Extras', 'Sale price',   
        'Sale date','Appraisal value', 'Appraisal date',  'Opinion of site value', 'Marketing time',  
        'Days on market', 'Prior sale date', 'Prior sale price', 'Latitud','Longitud','Zoning'
        ]

In [ ]:
dorado = dorado[cols]

In [ ]:
# id
dorado['Property id'] = dorado['Property id'].str.split('ID: ', expand=True)[1]
# address
dorado['Address/Unit'] = dorado['Street'] + ', ' + dorado['Address/Unit']
# price
for col in ['Sale price', 'Appraisal value','Opinion of site value','Prior sale price','GLA(feet)']:
    dorado[col] = dorado[col].str.replace('\$','').str.replace(',','')
    dorado[col] = dorado[col].replace({'':np.nan})
#for cols in ['Sale date']:
   # dorado[col] = pd.to_datetime(dorado[col])

In [ ]:
dorado['Sale date'] = pd.to_datetime(dorado['Sale date'])


In [ ]:
dorado.shape

In [ ]:
# to floats
for col in ['Bedrooms','Sale price', 'Appraisal value','Opinion of site value','Prior sale price'
           ,'Bathrooms','GLA(feet)']:
    dorado[col] = dorado[col].astype('float')

In [ ]:
#dorado.to_excel('Dorado_city_price_history.xlsx')

## comps

In [ ]:
comps_dorado = dorado[(dorado['Bedrooms'] >=3) & 
               (dorado['Bathrooms'] >=2) &
               (dorado['GLA(feet)'] >=2000) &
               (dorado['GLA(feet)'] <=5000) &
               (dorado['Sale price'] <=1000000)
              ]

In [ ]:
comps_dorado.shape

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=del_mar_full['Sale price'], xbins_size=50000, marker_color='blue', name='whole community'))
fig.add_trace(go.Histogram(x=comps_dorado['Sale price'], xbins_size=50000, marker_color='red', name='comps'))
fig.update_layout(width=800, height=350)
fig.show()

In [ ]:
comps_dorado['Sale date Year'] = comps_dorado['Sale date'].apply(lambda x: x.year)
comps_dorado['Sale date Quarter'] = comps_dorado['Sale date'].apply(lambda x: x.quarter)
comps_dorado['Sale date Year-Quarter'] = comps_dorado['Sale date Year'].astype(str) +' Q'+ comps_dorado['Sale date Quarter'].astype(str) 

### sales volume

In [ ]:
sales_volume_mar_comps = comps.groupby('Sale date Year-Quarter')['Sale price'].count()
sales_volume_mar_comps = pd.DataFrame(sales_volume_mar_comps ).rename(columns={'Sale price':'Volume'})

del_mar_full = del_mar_full.sort_values('Sale date Year-Quarter')
sales_volume_mar = del_mar_full.groupby('Sale date Year-Quarter')['Sale price'].count()
sales_volume_mar = pd.DataFrame(sales_volume_mar).rename(columns={'Sale price':'Volume'})

comps_dorado = comps_dorado.sort_values('Sale date Year-Quarter')
sales_volume_dorado_comps = comps_dorado.groupby('Sale date Year-Quarter')['Sale price'].count()
sales_volume_dorado_comps = pd.DataFrame(sales_volume_dorado_comps ).rename(columns={'Sale price':'Volume'})

#sales_volume['Sale price'] = [x/400*100 for x in sales_volume['Sale price']]

In [ ]:
sales_volume_mar_comps.index

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x= sales_volume_dorado_comps.index, 
                         y= sales_volume_dorado_comps['Volume'],
                         mode='markers',
                         name = 'Dorado (comps)',
                         marker=dict(size=10, opacity=0.6)
                           ))
fig.add_trace(go.Scatter(x= sales_volume_mar_comps.index, 
                         y= sales_volume_mar_comps['Volume'],
                         mode='markers',
                         name = 'Dorado del Mar (comps)',
                         marker=dict(size=10, opacity=0.6)
                           ))
fig.add_trace(go.Scatter(x= sales_volume_mar.index, 
                         y= sales_volume_mar['Volume'],
                         mode='markers',
                         name = 'Dorado del Mar',
                         marker=dict(size=10, opacity=0.6)
                           ))



fig.update_layout(title_text='Price per Sq.ft. Dorado del Mar Change 2008-2019',
                  width=1200, height=300, plot_bgcolor='rgba(0,0,0,0)', showlegend=True,
                  yaxis=dict(gridcolor='#EEEEEE', nticks=10),
                  xaxis=dict(tickangle=-45, gridcolor='#EEEEEE', 
                             tickfont=dict(size=9)))

fig.show()

In [ ]:
### sales volume

sales_volume_mar_comps = comps.groupby('Sale date Year-Quarter')['Sale price'].count()
sales_volume_mar_comps = pd.DataFrame(sales_volume_mar_comps ).rename(columns={'Sale price':'Volume'})

del_mar_full = del_mar_full.sort_values('Sale date Year-Quarter')
sales_volume_dorado = del_mar_full.groupby('Sale date Year-Quarter')['Sale price'].count()
sales_volume_dorado = pd.DataFrame(sales_volume_dorado).rename(columns={'Sale price':'Volume'})


#sales_volume['Sale price'] = [x/400*100 for x in sales_volume['Sale price']]

fig = go.Figure()
fig.add_trace(go.Scatter(x= sales_volume_mar_comps.index, 
                         y= sales_volume_mar_comps['Volume'],
                         mode='markers',
                         name = 'Dorado del Mar',
                         marker=dict(size=10, opacity=0.6)
                           ))
fig.add_trace(go.Scatter(x= sales_volume_dorado.index, 
                         y= sales_volume_dorado['Volume'],
                         mode='markers',
                         name = 'Dorado',
                         marker=dict(size=10, opacity=0.6)
                           ))



fig.update_layout(title_text='Price per Sq.ft. Dorado del Mar Change 2008-2019',
                  width=1200, height=300, plot_bgcolor='rgba(0,0,0,0)', showlegend=True,
                  yaxis=dict(gridcolor='#EEEEEE', nticks=10),
                  xaxis=dict(tickangle=-45, gridcolor='#EEEEEE', 
                             tickfont=dict(size=9)))

fig.show()

In [ ]:
comps_quarters = comps.groupby('Sale date Year-Quarter')['Sale price'].agg(Price_min='min', 
                                                                          Price_max='max', 
                                                                          Price_median='median')
comps_quarters = pd.DataFrame(comps_quarters)

In [ ]:
comps_quarters['Price_max'].sum()/comps_quarters.shape[0]

In [ ]:
min_change = [(comps_quarters['Price_min'][x+1]-comps_quarters['Price_min'][x])/comps_quarters['Price_min'][x]*100 for x in range(comps_quarters.shape[0]-1)]
percent_min = [0]
percent_min.extend(min_change)
max_change = [(comps_quarters['Price_max'][x+1]-comps_quarters['Price_max'][x])/comps_quarters['Price_max'][x]*100 for x in range(comps_quarters.shape[0]-1)]
percent_max = [0]
percent_max.extend(max_change)
comps_quarters['Price_max_change'] = percent_max
comps_quarters['Price_min_change'] = percent_min

In [ ]:
annotations_min=[set_annotation(k, comps_quarters['Price_min'][k], k, 'min') for k in range(comps_quarters.shape[0])]
annotations_max=[set_annotation(k, comps_quarters['Price_max'][k], k, 'max') for k in range(comps_quarters.shape[0])]
annotations_min.extend(annotations_max)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x= comps_quarters.index, 
                         y= comps_quarters['Price_min'],
                         mode='lines+markers',
                         marker_color='blue',
                         name = 'Min Quarter value',
                        ),
             )
fig.add_trace(go.Scatter(x= comps_quarters.index, 
                         y= comps_quarters['Price_max'],
                         mode='lines+markers',
                         marker_color='red',
                         name = 'Max Quarter value',
                         fill='tonexty',fillcolor='rgba(248, 245, 159, 0.1)'))
fig.add_trace(go.Scatter(x= comps_quarters.index, 
                         y= comps_quarters['Price_median'],
                         mode='lines',
                         name = 'Median Quarter value',
                         line = dict(shape = 'spline', width=1, dash='dashdot', color='grey')
                           ))
fig.update_layout(title_text='Price change Dorado Municipality 2008-2019',
                  width=1200, height=500, plot_bgcolor='rgba(0,0,0,0)', showlegend=True,
                  yaxis=dict(gridcolor='#EEEEEE', nticks=18),
                  xaxis=dict(tickangle=-45, gridcolor='#EEEEEE', 
                             tickfont=dict(size=9)))
fig.update_layout(annotations=annotations_min)
fig.show()

In [ ]:
import plotly.offline
plotly.offline.plot(fig, filename = 'filename.html', auto_open=False)

In [ ]:
comps['Price/area'] = comps['Sale price']/comps['GLA(feet)']

In [ ]:
comps_quarters = comps.groupby('Sale date Year-Quarter')['Price/area'].agg(Price_min='min', 
                                                                          Price_max='max', 
                                                                          Price_median='median')
comps_quarters = pd.DataFrame(comps_quarters)

In [ ]:
min_change = [(comps_quarters['Price_min'][x+1]-comps_quarters['Price_min'][x])/comps_quarters['Price_min'][x]*100 for x in range(comps_quarters.shape[0]-1)]
percent_min = [0]
percent_min.extend(min_change)
max_change = [(comps_quarters['Price_max'][x+1]-comps_quarters['Price_max'][x])/comps_quarters['Price_max'][x]*100 for x in range(comps_quarters.shape[0]-1)]
percent_max = [0]
percent_max.extend(max_change)
comps_quarters['Price_max_change'] = percent_max
comps_quarters['Price_min_change'] = percent_min

In [ ]:
annotations_min=[set_annotation(k, comps_quarters['Price_min'][k], k, 'min', percent=True) for k in range(comps_quarters.shape[0])]
annotations_max=[set_annotation(k, comps_quarters['Price_max'][k], k, 'max', percent=True) for k in range(comps_quarters.shape[0])]
annotations_min.extend(annotations_max)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x= comps_quarters.index, 
                         y= comps_quarters['Price_min'],
                         mode='lines+markers',
                         name = 'Min Quarter value'
                           ))
fig.add_trace(go.Scatter(x= comps_quarters.index, 
                         y= comps_quarters['Price_max'],
                         mode='lines+markers',
                         name = 'Max Quarter value',
                         fill='tonexty',fillcolor='rgba(248, 245, 159, 0.1)'
                           ))
fig.add_trace(go.Scatter(x= comps_quarters.index, 
                         y= comps_quarters['Price_median'],
                         mode='lines',
                         name = 'Median Quarter value',
                         line = dict(shape = 'spline', width=1, dash='dashdot', color='grey')
                           ))
fig.update_layout(title_text='Price per Sq.ft. Dorado Change 2008-2019',
                  width=1000, height=500, plot_bgcolor='rgba(0,0,0,0)', showlegend=True,
                  yaxis=dict(gridcolor='#EEEEEE', nticks=18),
                  xaxis=dict(tickangle=-45, gridcolor='#EEEEEE', 
                             tickfont=dict(size=9)))
fig.update_layout(annotations=annotations_min)


fig.show()

# Current Market

In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://www.realtor.com/realestateandhomes-detail/H-5-Medusa_Dorado_PR_00646_M94193-76867"
r = requests.get(url)
content = r.content
soup = BeautifulSoup(content, "html.parser")

In [ ]:
import cfscrape
from lxml import etree

In [ ]:
import requests
def telegram_bot_sendtext(bot_message,token,chatid):
    bot_token = token
    bot_chatID = chatid
    send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + bot_message
    response = requests.get(send_text)
    return response.json()['ok']

In [ ]:
telegram_key = '856344749:AAF5bInzIqtvWbcaeXSlZtLPys6E6zafcto'


In [ ]:
telegram_bot_sendtext("zdras'te", telegram_key,'188524609')


In [ ]:
header = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9',
          'Accept-Encoding': 'gzip, deflate, sdch',
          'Accept-Language' : 'nl-NL,nl;q=0.8,en-US;q=0.6,en;q=0.4',
          'Cache-Control' : 'max-age=0',
          'Connection': 'keep-alive',
          'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.81 Safari/537.36'}

In [ ]:
scraper = cfscrape.create_scraper()
url = "https://www.realtor.com/realestateandhomes-detail/G03-Calle-Madre-Perla_Dorado_PR_00646_M98687-17719"
scraped_html=scraper.get(url,headers=header).content
html = etree.HTML(scraped_html)

#### parsing

#### links

In [ ]:
url ='https://www.realtor.com/realestateandhomes-search/00646?pos=18.465155,-66.289702,18.486036,-66.251078,15'
scraped_html_main =scraper.get(url,headers=header).content
html_main = etree.HTML(scraped_html_main)
links_pre = html_main.xpath('//*[@id="srp-list"]/ul//@href')
links=[]
for i in range(0, len(links_pre)):
    
    if '/realestateandhomes' in links_pre[i] and links_pre[i] not in links:
        links.append(links_pre[i])

#### features

In [ ]:
features = html.xpath('//*[@id="ldp-property-meta"]/ul//text()')
main_features = []
for i in range(0, len(features)):
    features[i] = re.sub("\n+|\r+|\t+", "",  features[i])
    features[i] = re.sub("\s\s+|,", "",  features[i])
    if len(features[i]) != 0:
        main_features.append(features[i])
main_features = main_features[::2]
main_features #'beds', 'baths', 'living area', 'sqft lot'        

In [ ]:
features

In [ ]:
facts_pre = html.xpath('//*[@id="key-fact-carousel"]/ul//text()')
facts = []
for i in range(0, len(facts_pre)):
    facts_pre[i] = re.sub("\n+|\r+|\t+", "",  facts_pre[i])
    facts_pre[i] = re.sub("\s\s+", "",  facts_pre[i])
    if len(facts_pre[i]) != 0 and facts_pre[i] != '®':
        facts.append(facts_pre[i])
facts = facts[1::2]
facts #['Status', 'Price/Sq Ft', 'On realtor.com', 'Type', 'Built']        

In [ ]:
price = html.xpath('//*[@id="ldp-pricewrap"]/div/div/span/text()')
price = re.sub("\n+|\r+|\t+|\$|,", "",  price[0])
price = re.sub("\s\s+", "",  price)
price

In [ ]:
address_pre = html.xpath('//*[@id="ldp-address"]/h1//text()')
for i in range(0, len(address_pre)):
    address_pre[i] = re.sub("\n+|\r+|\t+", "",  address_pre[i])
    address_pre[i] = re.sub("\s\s+", "",  address_pre[i])
address = [x + ' ' for x in address_pre[1::2]]
address = ''.join(address)
address = re.sub("Dorado PR 00646 ", "Dorado, PR, 00646",  address)
address


In [ ]:
lot_descr = html.xpath('//*[@id="load-more-features"]/div[4]/div[1]/ul/li/text()')
lot_descr = re.findall('(?<=Lot Description: ).*', lot_descr[0])
lot_descr

In [ ]:
park_features = html.xpath('//*[@id="load-more-features"]/div[5]/div[1]/ul/li/text()')
park_features = re.findall('(?<=Parking Features: ).*', park_features[0])
park_features

In [ ]:
pre_total_park = html.xpath('//*[@id="load-more-features"]/div[5]/div[2]//text()')
total_park = []
for i in range(0, len(pre_total_park)):
    pre_total_park[i] = re.sub("\n+|\r+|\t+", "",  pre_total_park[i])
    pre_total_park[i] = re.sub("\s\s+", "",  pre_total_park[i])
    if len(pre_total_park[i]) != 0:
        pre_total_park[i] = re.findall('(?<=Parking Total: ).*', pre_total_park[i])[0]
        total_park.append(pre_total_park[i])
total_park

In [ ]:
coordinates = html.xpath('//*[@id="load-more-features"]/div[7]/div[1]/ul/li[2]/text()')
coordinates = re.findall('(?<=Directions: Coordinates: ).*', coordinates[0])[0].split(', ')
coordinates

In [ ]:
pd.DataFrame({'address':address,
              'price':price,
              'beds':main_features[0],
              'baths':main_features[1],
              'living area':main_features[2],
              'lot area':main_features[3],
              'extra':lot_descr[0],
              'status':facts[0],
              'days on market':facts[2],
              'property type':facts[3],
              'parking':park_features[0],
              'parking lots':total_park[0], 
              'year built':facts[4],
              'lat':coordinates[0],
              'lon':coordinates[1]
             }, index=[0])

#### *all in one place*

In [ ]:
from random import randint
from time import sleep

In [ ]:
table=pd.DataFrame([])
for n,j in enumerate(links[9:10]):
    url = 'https://www.realtor.com'+j
    print(url)
    sleep(randint(7,12))
    scraped_html=scraper.get(url,headers=header).content
    html = etree.HTML(scraped_html)
    
    features = html.xpath('//*[@id="ldp-property-meta"]/ul//text()')
    main_features = []
    for i in range(0, len(features)):
        features[i] = re.sub("\n+|\r+|\t+", "",  features[i])
        features[i] = re.sub("\s\s+|,", "",  features[i])
        if len(features[i]) != 0:
            main_features.append(features[i])
    main_features = main_features[::2]

    price = html.xpath('//*[@id="ldp-pricewrap"]/div/div/span/text()')
    if len(price) > 0:
        price = re.sub("\n+|\r+|\t+|\$|,", "",  price[0])
        price = re.sub("\s\s+", "",  price)
    else:
        price='-'


    address_pre = html.xpath('//*[@id="ldp-address"]/h1//text()')
    for i in range(0, len(address_pre)):
        address_pre[i] = re.sub("\n+|\r+|\t+", "",  address_pre[i])
        address_pre[i] = re.sub("\s\s+", "",  address_pre[i])
    address = [x + ' ' for x in address_pre[1::2]]
    address = ''.join(address)
    address = re.sub("Dorado PR 00646 ", "Dorado, PR, 00646",  address)
    
    row = pd.DataFrame({'address':address,
              'price':price,
              'beds':main_features[0],
              'baths':main_features[1],
              'living area':main_features[2],
              #'lot area':main_features[3],
              #'extra':lot_descr,
              #'status':facts[0],
              #'days on market':facts[2],
              #'property type':facts[3],
              #'parking':park_features,
              #'parking lots':total_park, 
              #'year built':facts[4],
              #'coordinates':coordinates,
              'url':url
              #'lon':coordinates[1]
             }, index=[n])
    
    table = pd.concat([table, row])

## Classificado

In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://www.clasificadosonline.com/UDREListing.asp?RESPueblos=&Category=%25&LowPrice=0&HighPrice=999999999&Bedrooms=%25&Area=Dorado+del+Mar&Submit2=Search+without+repossessed"
r = requests.get(url)
content = r.content
soup = BeautifulSoup(content, "html.parser")

In [ ]:
#table = soup.findAll(attrs={'class':'Ver14nounder'})
table = soup.findAll('table')

"""
for i in table:
    row = i.find("tr")
    for j in row:
        col = j.find("td")
        try:
            print(col.text)
            
        #for n in col:
            #print(n.text)
 """  

In [ ]:
table[43].findAll("tr")[0].findAll("td")[0].text